### Trying to figure out why tks's and my .0232 kernel are so good... and show how R measurements can differ a lot!

In [25]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import xgboost as xgb
import math

import time

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

import kagglegym

%pylab inline
import pylab as pl
pylab.rcParams['figure.figsize'] = (10, 6)

%matplotlib inline
import matplotlib.pyplot as plt

import os


Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [26]:
def rscore(y, y_hat, y_mean = None):
    ymean = y_mean if y_mean else np.mean(y)
    
    num = np.sum((y_hat - y) ** 2.0)
    denom = np.sum((y - ymean) ** 2.0) 

    ra = 1 - (num / denom)
    r = np.sign(ra) * np.sqrt(np.abs(1 - (num / denom)))

    return r

In [27]:
train = pd.read_hdf('../../twosigma/input/train.h5')

In [28]:
subs = {}

for f in os.listdir():
    if f[-4:] != '.pkl':
        continue
    
    name = f[:-4]
    
    print(name)
    
    subs[name] = pd.read_pickle(f)

sub-232017
kernel-sdfoley-results
sub-227738
sub-212725
kernel-tks-results


In [29]:
subs2 = {}

for s in subs.keys():
    ymean = subs[s].y.mean()

    scores = pd.DataFrame(columns=['day', 'r_day', 'r_all'])
    
    for g in subs[s].groupby('timestamp'):
        r_day = rscore(g[1].y, g[1].y_hat)
        r_all = rscore(g[1].y, g[1].y_hat, y_mean = ymean)
        
        scores.loc[len(scores)] = (g[0], r_day, r_all)
        
    subs2[s] = scores.copy()

In [68]:
subs_merge = pd.DataFrame(columns=['day'])

In [69]:
for s in subs2.keys():
    print(s, subs2[s].r_day.mean(), subs2[s].r_all.mean())

sub-232017 -0.13046021980854716 -0.0020613085262296688
kernel-sdfoley-results -0.12922836648276378 0.0013319721446613084
sub-227738 -0.12900475630503958 0.00393841143448099
sub-212725 -0.129474654647052 0.004040313185385112
kernel-tks-results -0.1275840320143353 0.004212891911545318


In [70]:
rename = {'sub-232017': '232017', 'kernel-sdfoley-results': 'sdfoley', 'sub-227738': '227738', 'kernel-tks-results': 'tks', 'sub-212725': '212725'}

In [71]:
subs_merge2 = subs2['kernel-tks-results'].copy()

In [72]:
subs_merge2

,day,r_day,r_all
0,906.0,-0.183311,0.026283
1,907.0,-0.059661,-0.015746
2,908.0,-0.120507,-0.086362
3,909.0,-0.136579,0.013083
4,910.0,-0.047105,0.030730
5,911.0,-0.127329,-0.048265
6,912.0,-0.335744,-0.021051
7,913.0,-0.028031,0.054133
8,914.0,-0.124021,-0.012643
9,915.0,-0.035447,-0.022003


In [73]:
for s in subs2.keys():
    if s == 'kernel-tks-results':
        continue
        
    subs_merge2 = pd.merge(subs_merge2, subs2[s], on='day', suffixes=['', '_'+rename[s]])

In [79]:
c = list(subs_merge2.columns)
c[1] = 'r_day_tks'
c[2] = 'r_all_tks'
subs_merge2.columns = c

In [80]:
subs_merge2

,day,r_day_tks,r_all_tks,r_day_232017,r_all_232017,r_day_sdfoley,r_all_sdfoley,r_day_227738,r_all_227738,r_day_212725,r_all_212725
0,906.0,-0.183311,0.026283,-0.203395,-0.082573,-0.200028,-0.074194,-0.202150,-0.079561,-0.189934,-0.041222
1,907.0,-0.059661,-0.015746,-0.102192,-0.084315,-0.072236,-0.043602,-0.048544,0.030838,-0.030153,0.048924
2,908.0,-0.120507,-0.086362,-0.112052,-0.074202,-0.070354,0.045246,-0.136509,-0.107436,-0.130978,-0.100366
3,909.0,-0.136579,0.013083,-0.115924,0.072733,-0.122671,0.060910,-0.119729,0.066408,-0.115499,0.073394
4,910.0,-0.047105,0.030730,-0.064683,-0.031850,-0.072465,-0.045590,-0.041521,0.037916,-0.040868,0.038618
5,911.0,-0.127329,-0.048265,-0.143146,-0.080927,-0.132084,-0.059549,-0.123194,-0.036163,-0.120597,-0.026137
6,912.0,-0.335744,-0.021051,-0.349328,-0.093862,-0.349178,-0.093361,-0.335796,-0.021784,-0.337298,-0.037197
7,913.0,-0.028031,0.054133,-0.081523,-0.053927,-0.045704,0.040399,-0.049566,0.035574,-0.055592,0.025183
8,914.0,-0.124021,-0.012643,-0.133517,-0.050681,-0.134558,-0.053326,-0.139938,-0.065562,-0.140375,-0.066474
9,915.0,-0.035447,-0.022003,-0.037944,-0.025830,-0.017317,0.021719,0.050304,0.057445,-0.017170,0.021836


In [95]:
#subs_merge2c = subs_merge2[np.logical_or(subs_merge2.day <= 905 + 500, subs_merge2.day >= 905 + 600)]
subs_merge2c = subs_merge2[np.logical_and(subs_merge2.day >= 905 + 500, subs_merge2.day <= 905 + 600)]

In [96]:
for c in subs_merge2.columns[1:]:
    if 'r_all' in c:
        print(c, subs_merge2[c].mean(), subs_merge2c[c].mean())

r_all_tks 0.004212891911545318 0.0015158759508956652
r_all_232017 -0.0020613085262296688 -0.0032566708393266656
r_all_sdfoley 0.0013319721446613084 0.00019939350193343513
r_all_227738 0.00393841143448099 0.004013876562024941
r_all_212725 0.004040313185385112 0.004491570084762808
